This program executes one simulation using initial parameters that were set in 'main_code.ipynb' and are saved and imported from 'initial_parameters.py'.

# Import libraries

In [1]:
#import hoomd that ist saved on computer
import sys
sys.path.append("/net/theorie/home/niklas.butkevich/miniconda2/envs/py3/lib/python3.7/site-packages")
import hoomd
import hoomd.md

import numpy as np
import math
import os

#import file with initial parameters
import initial_parameters

# Initialize the system

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


# Define the parameters (that were set in 'main_code.ipynb')

In [3]:
#define number of polymers segments (monomers)
Monomers = initial_parameters.Monomers

#define number of bonds between segments
Bonds = initial_parameters.Bonds

#define spring constant (Rouse model) for bonds between segments in units energy/distance^2
k = initial_parameters.k

#define number of passive particles
n_passive_particles = initial_parameters.n_passive_particles

#define number of active particles
n_active_particles = initial_parameters.n_active_particles

#define drag-coefficient constant gamma
gamma = initial_parameters.gamma

#define rest distance between segments
r0 = initial_parameters.r0

#define temperature in units of kT
kT = initial_parameters.kT

#define integration step time
dt = initial_parameters.dt

#define number of integration steps
integration_steps = initial_parameters.integration_steps

#define number of points for graphical representation (number of callbacks)
number_of_points = initial_parameters.number_of_points

#define period for analysing of quantities (write them in data file)
return_period = integration_steps/number_of_points

In [4]:
#set epsilon and sigma for Lennard_jones potential between different particles
lj_epsilon_AA = initial_parameters.lj_epsilon_AA
lj_sigma_AA = initial_parameters.lj_sigma_AA
lj_epsilon_AB = initial_parameters.lj_epsilon_AB
lj_sigma_AB = initial_parameters.lj_sigma_AB
lj_epsilon_BB = initial_parameters.lj_epsilon_BB
lj_sigma_BB = initial_parameters.lj_sigma_BB

#set cut-off radius for potential between two particles
r_cut = initial_parameters.r_cut

# Create box with particles

Attention: Box has to be large, otherwise - if polmer is outside the box it appears on the opposite side - the end-to-end-values and other position parameters are calculated in a wrong way.

In [5]:
#define box dimensions
Lx = 10 * Monomers
Ly = 10 * Monomers
Lz = 10 * Monomers

#define the system as snapshot
snapshot = hoomd.data.make_snapshot(N = Monomers + n_passive_particles,
                                    
                                    #box-dimension: x-direction: -Lx/2 to Lx/2 etc.
                                    box=hoomd.data.boxdim(Lx=Lx, Ly=Ly, Lz=Lz),
                                    
                                    #list of types; typeid[0]='A', typeid[1]='B', etc.
                                    particle_types=['A','B'],
                                    bond_types=['polymer'])

# Set ID's positions and bonds for monomers/particles

In [6]:
#create list to set monomer's/particle's initial type-IDs, positions, bonds
initial_IDs = []
initial_positions = []
initial_bonds= []

#define initial quantities
initial_position_of_cm_x = 0
initial_position_of_cm_y = 0
initial_position_of_cm_z = 0

#set initial monomer positions and IDs
for monomer in range(Monomers):
    initial_IDs.append(0)
    initial_positions.append([monomer - (Monomers/2) + 0.5, 0, 0])
    
    #calculate initial distance of center of mass from origin
    initial_position_of_cm_x += (initial_positions[monomer][0] / Monomers)
    initial_position_of_cm_y += (initial_positions[monomer][1] / Monomers)
    initial_position_of_cm_z += (initial_positions[monomer][2] / Monomers)
    
initial_position_of_cm = [initial_position_of_cm_x,
                          initial_position_of_cm_y,
                          initial_position_of_cm_z]

initial_end_to_end_vector = [initial_positions[Monomers-1][0]-initial_positions[0][0],
                             initial_positions[Monomers-1][1]-initial_positions[0][1],
                             initial_positions[Monomers-1][2]-initial_positions[0][2]]
    
for bond in range(Monomers-1):
    initial_bonds.append([bond, bond+1])
    
#resize the bonds list to actual number of bonds
snapshot.bonds.resize(Monomers-1)

#set initial passive particle positions and IDs
for passive_particle in range(n_passive_particles):
    initial_IDs.append(1)
    initial_positions.append([passive_particle - (n_passive_particles/2) + 0.5, 1, 0])
    
#copy IDs, positions and bonds into valid arrays
snapshot.particles.typeid[:] = initial_IDs
snapshot.particles.position[:] = initial_positions
snapshot.bonds.group[:] = initial_bonds

# Initialization of integration parameters

In [7]:
#initialize HOOMD using snapshot
system = hoomd.init.read_snapshot(snapshot)

#set the bond type and strength between monomers
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=k, r0=r0)

#consider every particle
all = hoomd.group.all()

#safe the trajectory in 'results/datetime' to
#display in a visualisation program

hoomd.dump.gsd("results/"+str(initial_parameters.current_datetime)+
               "/trajectory.gsd",
               
               #return_period = number of points in plots
               period=return_period,
               group=all, overwrite=True)"""

notice(2): Group "all" created containing 50 particles


'hoomd.dump.gsd("results/"+str(initial_parameters.current_datetime)+\n               "/trajectory.gsd",\n               \n               #return_period = number of points in plots\n               period=return_period,\n               group=all, overwrite=True)'

# Set integrator and  parameters

In [8]:
#integrator
hoomd.md.integrate.mode_standard(dt=dt)
integrator = hoomd.md.integrate.brownian(group=all, kT=kT, dscale=False,
                                         seed=np.random.randint(0,9999))


#set gamma (friction constant); ATTENTION: set after integrator is set!
integrator.set_gamma('A', gamma=gamma)


notice(2): integrate.langevin/bd is using specified gamma values


# Load summen quantities that were calculated earlier

In [9]:
#open quantities file
quantities_file = open('quantities.dat','r')

#load all lines, begin with second line (1.line is description)
#now, 'quantities.dat'-file can be overwritten in the run process
all_lines = quantities_file.readlines()[1:]

#close file
quantities_file.close()

# Create classes to compute quantities

Note: For every run of THIS entire code ('run_polymer_simulation_once.ipynb') the current values for quantities are added to values calculated earlier. The division by the number of execution of THIS file is done by the 'main_code.ipynb'.

In [10]:
#create class for end-to-end distance
class Sq_end_to_end_distance:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        old_sq_end_to_end_distance = float(all_lines[int(timestep/return_period)].split('	')[1])
        
        #define positions of first and last monomer
        m_first = snapshot.particles.position[0]
        m_last = snapshot.particles.position[-1]
        
        #define positions of first and last monomer
        current_sq_end_to_end_distance = ((m_last[0]-m_first[0])**2 +
                                          (m_last[1]-m_first[1])**2 +
                                          (m_last[2]-m_first[2])**2)
        
        #add current sq_end_to_end_distance
        summed_sq_end_to_end_distance = old_sq_end_to_end_distance + current_sq_end_to_end_distance
        
        #return summed end-to-end distance
        return (summed_sq_end_to_end_distance)

In [11]:
#create class for distance of centre of mass from origin
class Sq_distance_of_cm:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        #old distance of centre of mass from origin (regarding to certain time step)
        old_sq_distance_of_cm = float(all_lines[int(timestep/return_period)].split('	')[2])
        
        #define lists of x,y and z positions of monomers
        list_of_x_positions = []
        list_of_y_positions = []
        list_of_z_positions = []
        
        #compute current position of centre of mass (the average is done by other Python-program)
        for monomers in range(Monomers):
            
            list_of_x_positions.append(snapshot.particles.position[monomers][0])
            list_of_y_positions.append(snapshot.particles.position[monomers][1])
            list_of_z_positions.append(snapshot.particles.position[monomers][2])
            
        current_position_of_cm = [np.mean(list_of_x_positions),
                                  np.mean(list_of_y_positions),
                                  np.mean(list_of_z_positions)]
        
        #the initial position of centre mass is [0,0,0], therefore the distance to this origin
        #can be calculated in a trivial way
        
        current_sq_distance_of_cm = ((current_position_of_cm[0]-initial_position_of_cm[0])**2 +
                                     (current_position_of_cm[1]-initial_position_of_cm[1])**2 +
                                     (current_position_of_cm[2]-initial_position_of_cm[2])**2)
        
        summed_sq_distance_of_cm = old_sq_distance_of_cm + current_sq_distance_of_cm
        
        return (summed_sq_distance_of_cm)

In [12]:
#create class for auto correlation of end-to-end distance vector
class Auto_corr_ee_vector:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        snapshot = self.system.take_snapshot()
        
        old_auto_corr_ee_vector = float(all_lines[int(timestep/return_period)].split('	')[3])
        
        end_to_end_vector_of_t = [(snapshot.particles.position[-1][0] -
                                   snapshot.particles.position[0][0]),
                                  
                                  (snapshot.particles.position[-1][1] -
                                   snapshot.particles.position[0][1]),
                                  
                                  (snapshot.particles.position[-1][2] -
                                   snapshot.particles.position[0][2])]
        
        auto_corr_ee_vector = (end_to_end_vector_of_t[0] * initial_end_to_end_vector[0] +
                               end_to_end_vector_of_t[1] * initial_end_to_end_vector[1] +
                               end_to_end_vector_of_t[2] * initial_end_to_end_vector[2])
        
        summed_auto_corr_ee_vector = old_auto_corr_ee_vector + auto_corr_ee_vector
        
        return (summed_auto_corr_ee_vector)

# Create instances for classes and log the quantities

In [13]:
instance_sq_end_to_end_distance = Sq_end_to_end_distance(system)
instance_sq_distance_of_cm      = Sq_distance_of_cm(system)
instance_auto_corr_ee_vector    = Auto_corr_ee_vector(system)

#use analyze.log to write quantities of simulation into a text file
logger = hoomd.analyze.log(filename='quantities.dat',
                           quantities=['summed_sq_end_to_end_distance',
                                       'summed_sq_distance_of_cm',
                                       'summed_auto_corr_ee_vector'],
                           
                           #return and write to file
                           period = return_period,
                           header_prefix='#',
                           overwrite=True)

#create a new quantity that is logged due to expression above to a text file
logger.register_callback(('summed_sq_end_to_end_distance'), instance_sq_end_to_end_distance)
logger.register_callback(('summed_sq_distance_of_cm'), instance_sq_distance_of_cm)
logger.register_callback(('summed_auto_corr_ee_vector'), instance_auto_corr_ee_vector)

# Run the simulation

In [14]:
hoomd.run(integration_steps)

** starting run **
Time 00:00:02 | Step 100000 / 100000 | TPS 39454.5 | ETA 00:00:00
Average TPS: 39448.8
---------
** run complete **
